# Towards reproducible analysis of benthos structural complexity: A case study on Antarctic polychaete reefs using action cameras and remotely operated vehicles
- Aim: Relationships between substrate type and structural complexity
- Goal: Create a complete data pipeline for data analysis and visualization
- Authors: J.C. Montes-Herrera, G. Johnstone, J. Stark, N. Hill, V. Cummings, V. Lucieer
- Contact: juancarlos.montesherrera@utas.edu.au
- Associated publication submitted to journal Remote Sensing in Ecology & Conservation.

## Step 1. Generate 1 x 1 meter quadrat raster files with GDAL
- Create new folder called \rasters and download data https://doi.org/10.5281/zenodo.7115132
- Create output folder for quadrat file called \quadrats
- We assume files were exported in the following resolution.
    - Orthomosaic: 0.5 mm/pix
    - DEM: 1 cm/pix

In [ ]:
## Libraries
import os
import glob
from osgeo import gdal

In [ ]:
## Create a Path to the data
PATH_d = r"C:\Users\jcmontes\OneDrive - University of Tasmania\01_Projects_Drive\Photogrammetry\AAD2019_EF_PolychaeteReef\processed_data\EF-G-AggressiveFilter\EF-G-PolygonExport - Copy"

os.chdir(PATH_d)

dems = []
orthos = []

for file in glob.glob('*DEM*.tif'):
    dems.append(file)

for file in glob.glob('*ortho*.tif'):
    orthos.append(file)

In [ ]:
## Prepare PATH to generate output
PATH_o = r"C:\Users\jcmontes\Documents\GitHub\ant_biogenic_structures\data\quadrats/"

In [ ]:
# Process Digital Elevation Models (1 cm/pixel resolution)
for q, raster in enumerate(dems):
    dem = gdal.Open(dems[q])

    gt = dem.GetGeoTransform()

    xmin = gt[0]
    ymax = gt[3]
    res = gt[1]

    # raster lengths
    xlen = res * dem.RasterXSize
    ylen = res * dem.RasterYSize

    # number of tiles in x and y direction
    xdiv = int(xlen)
    ydiv = int(ylen)

    # size of single tile
    xsize = xlen/xdiv
    ysize = ylen/ydiv

    # create lists of x and y coordinates
    xsteps = [xmin + xsize * i for i in range(xdiv+1)]
    ysteps = [ymax - ysize * i for i in range(ydiv+1)]

    for i in range(xdiv):
        for j in range(ydiv):
            xmin = xsteps[i]
            xmax = xsteps[i+1]
            ymax = ysteps[j]
            ymin = ysteps[j+1]

            # gdal translate to subset the input raster
            gdal.Translate(PATH_o + str(dems[q][:-4]) + "_"+str(i)+str(j)+".tif", dem, projWin = (xmin, ymax, xmax, ymin), xRes = res, yRes = -res)

    #close the open dataset to flush memory
    dem = None
    

# Now the orthomosaic (0.5 cm/pixel resolution)
for q, raster in enumerate(orthos):
    ortho = gdal.Open(orthos[q])

    gt = ortho.GetGeoTransform()

    xmin = gt[0]
    ymax = gt[3]
    res = gt[1]

    # raster lengths
    xlen = res * ortho.RasterXSize
    ylen = res * ortho.RasterYSize

    # number of tiles in x and y direction
    xdiv = int(xlen)
    ydiv = int(ylen)

    # size of single tile
    xsize = xlen/xdiv
    ysize = ylen/ydiv

    # create lists of x and y coordinates
    xsteps = [xmin + xsize * i for i in range(xdiv+1)]
    ysteps = [ymax - ysize * i for i in range(ydiv+1)]

    for i in range(xdiv):
        for j in range(ydiv):
            xmin = xsteps[i]
            xmax = xsteps[i+1]
            ymax = ysteps[j]
            ymin = ysteps[j+1]

            # gdal translate to subset the input raster
            gdal.Translate(PATH_o + str(orthos[q][:-4]) + "_"+str(i)+str(j)+".tif", ortho, projWin = (xmin, ymax, xmax, ymin), xRes = res, yRes = -res)

    #close the open dataset to flush memory
    ortho = None